# A Test for the Tab App

This uses [importnb](https://pypi.org/project/importnb) to load the
[Tab App Example](./Tab%20App%20Example.ipynb) for interative and automated testing.

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
from requests_cache import CachedSession

with importnb.Notebook():
    try:
        from Tab_App_Example import tabs, W
    except ImportError:
        from .Tab_App_Example import tabs, W

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Example data is loaded from [schema.org](https://schema.org/docs/developers.html).

In [ ]:
TEST_URL = "https://raw.githubusercontent.com/schemaorg/schemaorg/v9.0-release/data/releases/8.0/schema.{fmt}"

Tests are implemented as buttons you can click.

In [ ]:
tests = [
    W.Button(description=f"{fmt}") for fmt in ["ttl", "nt", "jsonld", "rdf", "rdfa"]
]
test_log = W.Output()
with test_log:
    print("click a button to start a test")
tabs.layout.width = "100%"

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
def _run_test(btn):
    test_log.clear_output()
    session = CachedSession(".cache")
    fmt = btn.description
    errors = []
    try:
        with test_log:
            timestamp(fmt, "starting...")
            tabs.selected_index = 0
            url = TEST_URL.format(fmt=fmt)
            timestamp(fmt, f"fetching {url}...")
            response = session.get(url)
            timestamp(fmt, f"... response: {response.status_code}")
            assert response.status_code == 200
            timestamp(fmt, f"... {len(response.text)} bytes")
            timestamp(fmt, f"loading...")
            tabs.load_widget.load_box.file_upload_value = {
                Path(url).name: {
                    "metadata": {"length": len(response.text)},
                    "content": response.text,
                }
            }
            timestamp(fmt, f"... {len(tabs.load_widget.graph)} triples loaded")
            assert len(tabs.load_widget.graph)
            tabs.selected_index = 1
            timestamp(fmt, f"[{fmt}] querying...")
            tabs.query_widget.run_button.click()
            tabs.selected_index = 2
            timestamp(fmt, "OK!")
    except Exception as err:
        errors += [errors]
        timestamp(fmt, "ERROR")
        timestampe(fmt, err)


[d.on_click(_run_test) for d in tests]

## Show the Test Application

In [ ]:
W.VBox([tabs, W.HBox([W.Label("Start Test"), *tests]), test_log])

## (Maybe) Run the test automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()
        Path(f"Test Tab App.{test.description}.log").write_text(
            json.dumps(test_log.outputs, indent=2, sort_keys=True)
        )